In [2]:
import pandas as pd
import requests
import io

In [3]:
url = "https://raw.githubusercontent.com/JenPink25/Ames-Iowa-Housing-Dataset/master/data/ames%20iowa%20housing.csv"

# Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [4]:
df = df.drop('Id', axis='columns')

In [26]:
def transforms_features(df):
    '''clean data'''
    
    # remove columns with more than 25% of missing data
    cutoff_25 = .25*len(df)
    null_count_cols = df.isnull().sum()
    df = df[null_count_cols[null_count_cols <= cutoff_25].index]
    
    # drop columns that have missing text values
    missing_text = df.select_dtypes(include=['object']).isnull().sum()
    drop_missing_text = missing_text[missing_text > 0]
    df = df.drop(drop_missing_text.index, axis=1)
    
    # fill missing values with most popular value for numeric when the column has less than 5% of missing data
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)*.05) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    # feature engineering
#     years_sold = df['YrSold'] - df['Year Built']
#     years_since_remod = df['YrSold'] - df['Year Remod/Add']
#     df['Years Before Sale'] = years_sold
#     df['Years Since Remod'] = years_since_remod
#     df = df.drop([1702, 2180, 2181], axis=0)

    # categorical variables
    # cat_vars = ["Sale Condition", "Sale Type", "Mo Sale"]
    
    # drop columns
   # df = df.drop(["PID", "Order", "Year Built", "Year Remod/Add"], axis=1)
    return df

In [20]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [27]:
df = transforms_features(df)